📝 **Author:** Amirhossein Heydari - 📧 **Email:** <amirhosseinheydari78@gmail.com> - 📍 **Origin:** [mr-pylin/media-processing-workshop](https://github.com/mr-pylin/media-processing-workshop)

---


**Table of contents**<a id='toc0_'></a>    
- [Dependencies](#toc1_)    
- [Signals and Systems](#toc2_)    
  - [Linear Time-Invariant (LTI) Systems](#toc2_1_)    
    - [Convolution](#toc2_1_1_)    
      - [Circular Convolution](#toc2_1_1_1_)    
      - [Separable Convolution](#toc2_1_1_2_)    
  - [Cross-Correlation](#toc2_2_)    
  - [Padding](#toc2_3_)    
  - [Examples](#toc2_4_)    
    - [Example 1: Apply Padding to Signals](#toc2_4_1_)    
      - [Using NumPy](#toc2_4_1_1_)    
      - [Using OpenCV](#toc2_4_1_2_)    
    - [Example 2: 1D Convolution](#toc2_4_2_)    
      - [Manual](#toc2_4_2_1_)    
      - [Using NumPy](#toc2_4_2_2_)    
      - [Using SciPy](#toc2_4_2_3_)    
    - [Example 3: 2D Convolution](#toc2_4_3_)    
      - [Manual](#toc2_4_3_1_)    
      - [Using OpenCV](#toc2_4_3_2_)    
      - [Using SciPy](#toc2_4_3_3_)    
    - [Example 4: Separable Convolution](#toc2_4_4_)    
      - [Manual](#toc2_4_4_1_)    
    - [Example 5: 2D Cross-Correlation](#toc2_4_5_)    
      - [Using SciPy](#toc2_4_5_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Dependencies](#toc0_)


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from numpy.typing import NDArray

# <a id='toc2_'></a>[Signals and Systems](#toc0_)

- A **signal** is a function that conveys information about the behavior or attributes of a **physical phenomenon**.
  - **Continuous-time**: $x(t)$, defined for every instant of time.
  - **Discrete-time**: $x[n]$, defined only at specific instants of time.
- A **system** is an entity that processes an input signal to produce an output signal.
  - **Continuous-time**:
    $$y(t) = T\{x(t)\}$$
  - **Discrete-time**:
    $$y[n] = T\{x[n]\}$$

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/signal-and-system.svg" alt="signal-and-system.svg" style="max-width:80%; height:auto;">
  <figcaption>Signals and Systems</figcaption>
</figure>

📋 **Properties of Systems:**

1. **Linearity**: A system is linear if it satisfies the **superposition principle**:
   $$T\{a x_1(t) + b x_2(t)\} = a T\{x_1(t)\} + b T\{x_2(t)\}$$
1. **Time-Invariance**: A system is time-invariant if a time shift in the **input** results in the same time shift in the **output**:
   $$y(t - t_0) = T\{x(t - t_0)\}$$
1. **Causality**: A system is causal if the **output** at any time $\mathbf{t}$ depends only on the input at the **present** and **past** times, **not future** times.
1. **Stability**: A system is stable if bounded inputs produce bounded outputs (**BIBO stability**).
1. **Memory**: A system is **memoryless** if the output at any time $\mathbf{t}$ depends only on the input at the same time $\mathbf{t}$.

📝 **Docs**:

- Signal processing: [en.wikipedia.org/wiki/Signal_processing](https://en.wikipedia.org/wiki/Signal_processing)


## <a id='toc2_1_'></a>[Linear Time-Invariant (LTI) Systems](#toc0_)

- An **LTI** system is a system that is both **linear** and **time-invariant**.
- The **impulse response** of an LTI system is the output of the system when the input is an **impulse**:
  - For **continuous-time systems**:
    $$h(t) = T\{\delta(t)\}$$
  - For **discrete-time systems**:
    $$h[n] = T\{\delta[n]\}$$

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/impulse.svg" alt="impulse.svg" style="max-width:80%; height:auto;">
  <figcaption>Impulse Signal</figcaption>
</figure>

- The impulse response $\mathbf{h(t)}$ or $\mathbf{h[n]}$ completely characterizes an LTI system.
- Once the **impulse response** is known, the **output** for **any input** can be computed using **Convolution**.

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/lti-system.svg" alt="lti-system.svg" style="max-width:80%; height:auto;">
  <figcaption>Linear Time-Invariant Systems</figcaption>
</figure>

❓ **Example:**

- $h[n] = \{1, 2, 1\}$
- $x[n] = \{1, 2\}$
- $y[n] = ?$

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/lti-example.svg" alt="lti-example.svg" style="max-width:100%; height:auto;">
  <figcaption>Linear Time-Invariant Systems Example</figcaption>
</figure>

📝 **Docs**:

- Linear time-invariant system: [en.wikipedia.org/wiki/Linear_time-invariant_system](https://en.wikipedia.org/wiki/Linear_time-invariant_system)


### <a id='toc2_1_1_'></a>[Convolution](#toc0_)

- Convolution is a mathematical operation that **combines** two functions to produce a third function.
- For **LTI** systems, convolution is used to compute the **output** of the system given the **input** and the **impulse response**.
- **Continuous-Time Convolution**
  - For continuous-time LTI systems, the output $\mathbf{y(t)}$ is the convolution of the input $\mathbf{x(t)}$ and the impulse response $\mathbf{h(t)}$:
  $$y(t) = x(t) * h(t) = \int_{-\infty}^{\infty} x(\tau) h(t - \tau) \, d\tau$$
  - This **integral** represents the superposition of **scaled** and **shifted** versions of the **impulse response**.
- **Discrete-Time Convolution**
  - For discrete-time LTI systems, the output $\mathbf{y[n]}$ is the convolution of the input $\mathbf{x[n]}$ and the impulse response $\mathbf{h[n]}$:
  $$y[n] = x[n] * h[n] = \sum_{k=-\infty}^{\infty} x[k] h[n - k]$$
  - This **sum** represents the weighted **sum** of **shifted** versions of the **impulse response**.

❓ **Example:**

- $h[n] = \{1, 2, 1\}$
- $x[n] = \{1, 2\}$
- $y[n] = ?$

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/convolution-kernel-flip.svg" alt="convolution-kernel-flip.svg" style="max-width:80%; height:auto;">
  <figcaption>Convolution (flipped kernel)</figcaption>
</figure>

📋 **Properties of Convolution:**

1. **Commutativity**:
    $$x[n] * h[n] = h[n] * x[n]$$
1. **Associativity**:
    $$(x(t) * h_1(t)) * h_2(t) = x(t) * (h_1(t) * h_2(t))$$
1. **Distributivity**:
    $$x(t) * (h_1(t) + h_2(t)) = x(t) * h_1(t) + x(t) * h_2(t)$$
1. **Identity**:
    $$x(t) * \delta(t) = x(t)$$

📝 **Docs**:

- Convolution: [en.wikipedia.org/wiki/Convolution](https://en.wikipedia.org/wiki/Convolution)
- `numpy.convolve`: [numpy.org/doc/2.1/reference/generated/numpy.convolve.html](https://numpy.org/doc/2.1/reference/generated/numpy.convolve.html)
- `scipy.signal`: [docs.scipy.org/doc/scipy/reference/signal.html](https://docs.scipy.org/doc/scipy/reference/signal.html)
- `cv2.filter2D`: [docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04](https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga27c049795ce870216ddfb366086b5a04)


#### <a id='toc2_1_1_1_'></a>[Circular Convolution](#toc0_)

- Circular convolution is a special type of convolution where the input signals are assumed to be **periodic**.
- For two discrete signals $\mathbf{x[n]}$ and $\mathbf{h[n]}$ of length $\mathbf{N}$, the circular convolution is defined as:

$$y[n] = \sum_{m=0}^{N-1} x[m] h[(n - m) \mod N]$$

- The result is another **finite-length** sequence of the **same length** as the **inputs**.

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/circular-convolution.svg" alt="circular-convolution.svg" style="max-width:90%; height:auto;">
  <figcaption>Circular Convolution</figcaption>
</figure>

- It is particularly important in the context of **Fourier transforms**, as **convolution** in the **time domain** corresponds to **multiplication** in the **frequency domain**:

$$\mathcal{F}\{x[n] * h[n]\} = X(\omega) \cdot H(\omega)$$
$$\mathcal{F}\{x[n] \cdot h[n]\} = X(\omega) * H(\omega)$$

🆚 **Circular vs. Linear Convolution**:

- **Linear Convolution**: Used for finite non-periodic signals. $\text{length} = N+M−1$.
- **Circular Convolution**: Assumes periodicity. $\text{length} = max(N,M)$.
- **Key Insight**: Circular convolution **wraps around overlapping parts**, while linear convolution **zero-pads**.

🤚 **Avoiding Wrap-Around Artifacts**:

- To compute **linear convolution** using **DFT**:
  - Zero-pad both sequences to length $N+M−1$.
  - Perform circular convolution on the padded sequences.

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/linear-vs-circular-convolution.svg" alt="linear-vs-circular-convolution.svg" style="max-width:90%; height:auto;">
  <figcaption>Linear vs. Circular Convolution</figcaption>
</figure>

🔢 **Matrix Representation (Circulant Matrix)**:

- Circular convolution can be represented as multiplication by a **circulant matrix**.
- For $h=[h_0​,h_1​,h_2​]$:

$$
\begin{bmatrix}
h_0 & h_2 & h_1 \\
h_1 & h_0 & h_2 \\
h_2 & h_1 & h_0 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\end{bmatrix}
=
\begin{bmatrix}
y_1 \\
y_2 \\
y_3 \\
\end{bmatrix}
$$


#### <a id='toc2_1_1_2_'></a>[Separable Convolution](#toc0_)

- Separable convolution is a technique used to **reduce computation** when applying **2D** convolution filters.
- A convolution kernel $\mathbf{K}$ is **separable** if it can be **decomposed** into the **outer product** of two **1D kernels** (its matrix has **Rank 1**):

\begin{aligned}
K &= k_c \otimes k_r \\
&\text{where } k_r \text{ is a 1D row filter and } k_c \text{ is a 1D column filter.}
\end{aligned}

- Instead of performing a full 2D convolution, we can apply a **row-wise** convolution first, followed by a **column-wise** convolution.
- This reduces complexity from $\mathcal{O}(n^2 k^2)$ to $\mathcal{O}(n^2 k) + \mathcal{O}(n^2 k) = \mathcal{O}(2n^2 k)$ per dimension.

**Example**:

$$
S_x = 
\begin{bmatrix}
1 & 0 & -1 \\
2 & 0 & -2 \\
1 & 0 & -1
\end{bmatrix} 
= 
\underbrace{\begin{bmatrix} 1 \\ 2 \\ 1 \end{bmatrix}}_{k_c} 
\cdot 
\underbrace{\begin{bmatrix} 1 & 0 & -1 \end{bmatrix}}_{k_r}
$$

For example, if \( k = 3 \):

$$
1 - \frac{2}{3} \approx 33.3\% \text{ reduction in operations.}
$$


## <a id='toc2_2_'></a>[Cross-Correlation](#toc0_)

- It measures the **similarity** between two signals.
- It is closely related to convolution, but unlike convolution, cross-correlation **DOES NOT** flip the kernel.
- This makes cross-correlation particularly useful in applications like **template matching**, **signal alignment**, and **feature detection**.

$$y[n] = \sum_{k} x[k] \cdot h[k + n]$$
$$y[i, j] = \sum_{m} \sum_{n} x[m, n] \cdot h[m + i, n + j]$$

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/convolution-vs-correlation.svg" alt="convolution-vs-correlation.svg" style="max-width:80%; height:auto;">
  <figcaption>Convolution vs. Cross-Correlation</figcaption>
</figure>

📝 **Docs**:

- Cross-correlation: [en.wikipedia.org/wiki/Cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation)
- `scipy.signal`: [docs.scipy.org/doc/scipy/reference/signal.html](https://docs.scipy.org/doc/scipy/reference/signal.html)


## <a id='toc2_3_'></a>[Padding](#toc0_)

Padding is used to control the **size** of the **output** after applying a **convolution** or **cross-correlation** operation.

<table style="margin:0 auto;">
  <thead>
    <tr>
      <th>Category</th>
      <th>Type</th>
      <th>Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td rowspan="5"><strong>Padded Values</strong></td>
      <td>Zero Padding</td>
      <td>Adds zeros around the input signal.</td>
    </tr>
    <tr>
      <td>Reflect Padding</td>
      <td>Mirrors the input signal at the boundaries (excluding the boundary value).</td>
    </tr>
    <tr>
      <td>Symmetric  Padding</td>
      <td>Mirrors the input signal at the boundaries (including the boundary value).</td>
    </tr>
    <tr>
      <td>Circular (Wrap) Padding</td>
      <td>Wraps the input signal around as if it were periodic.</td>
    </tr>
    <tr>
      <td>Replicate (Edge) Padding</td>
      <td>Repeats the edge values of the input signal.</td>
    </tr>
    <tr>
      <td rowspan="3"><strong>Amount of Padding</strong></td>
      <td>Valid Padding (No Padding)</td>
      <td>No padding is added. The output size is smaller than the input size.</td>
    </tr>
    <tr>
      <td>Same Padding</td>
      <td>Adds padding such that the output size matches the input size.</td>
    </tr>
    <tr>
      <td>Full Padding</td>
      <td>Adds enough padding so that every element of the input is covered by the kernel at least once. The output size is larger than the input size.</td>
    </tr>
  </tbody>
</table>

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/padding-1d.svg" alt="padding-1d.svg" style="max-width:90%; height:auto;">
  <figcaption>Padding for 1D Convolution</figcaption>
</figure>

<figure style="text-align:center; margin:0;">
  <img src="../../assets/images/original/vector/lti/padding-2d.svg" alt="padding-2d.svg" style="max-width:80%; height:auto;">
  <figcaption>Padding for 2D Convolution</figcaption>
</figure>

📝 **Docs**:

- `numpy.pad`: [numpy.org/doc/stable/reference/generated/numpy.pad.html](https://numpy.org/doc/stable/reference/generated/numpy.pad.html)
- `cv2.copyMakeBorder`: [docs.opencv.org/master/d2/de8/group__core__array.html#ga2ac1049c2c3dd25c2b41bffe17658a36](https://docs.opencv.org/master/d2/de8/group__core__array.html#ga2ac1049c2c3dd25c2b41bffe17658a36)


## <a id='toc2_4_'></a>[Examples](#toc0_)


### <a id='toc2_4_1_'></a>[Example 1: Apply Padding to Signals](#toc0_)


In [ ]:
signal_1d = np.array([1, 3, 2, 1, 2], dtype=np.float32)
signal_2d = np.array([[1, 2, 3], [2, 1, 1], [3, 1, 2]], dtype=np.float32)

# log
print(f"signal_1d:\n{signal_1d}\n")
print(f"signal_2d:\n{signal_2d}")

#### <a id='toc2_4_1_1_'></a>[Using NumPy](#toc0_)


In [ ]:
signal_1d_1 = np.pad(signal_1d, pad_width=(2, 2), mode="constant", constant_values=0)
signal_1d_2 = np.pad(signal_1d, pad_width=(2, 2), mode="reflect")
signal_1d_3 = np.pad(signal_1d, pad_width=(2, 2), mode="symmetric")
signal_1d_4 = np.pad(signal_1d, pad_width=(2, 2), mode="wrap")
signal_1d_5 = np.pad(signal_1d, pad_width=(2, 2), mode="edge")

# plot
padded_signals = [signal_1d_1, signal_1d_2, signal_1d_3, signal_1d_4, signal_1d_5]
titles = ["Constant", "Reflect", "Symmetric", "Wrap", "Edge"]
fig, axes = plt.subplots(5, 1, figsize=(5, 5), layout="compressed")
for ax, signal, title in zip(axes, padded_signals, titles):
    ax.imshow(np.expand_dims(signal, axis=0), cmap="gray", vmin=0, vmax=3, aspect="auto")
    ax.set(title=title, xticks=range(signal.shape[0]), yticks=[])
    rect = plt.Rectangle((1.5, -0.5), 5, 1, edgecolor="red", linewidth=3, fill=False)
    ax.add_patch(rect)
plt.show()

#### <a id='toc2_4_1_2_'></a>[Using OpenCV](#toc0_)


In [ ]:
signal_2d_1 = cv2.copyMakeBorder(signal_2d, 2, 2, 2, 2, cv2.BORDER_CONSTANT, value=0)
signal_2d_2 = cv2.copyMakeBorder(signal_2d, 2, 2, 2, 2, cv2.BORDER_REFLECT)
signal_2d_3 = cv2.copyMakeBorder(signal_2d, 2, 2, 2, 2, cv2.BORDER_REFLECT_101)
signal_2d_4 = cv2.copyMakeBorder(signal_2d, 2, 2, 2, 2, cv2.BORDER_WRAP)
signal_2d_5 = cv2.copyMakeBorder(signal_2d, 2, 2, 2, 2, cv2.BORDER_REPLICATE)

# plot
padded_images = [signal_2d_1, signal_2d_2, signal_2d_3, signal_2d_4, signal_2d_5]
titles = ["Constant", "Reflect", "Symmetric", "Wrap", "Replicate"]
fig, axes = plt.subplots(1, 5, figsize=(15, 5), layout="compressed")
for ax, img, title in zip(axes, padded_images, titles):
    ax.imshow(img, cmap="gray", vmin=0, vmax=3)
    ax.set(title=title, xticks=range(img.shape[1]), yticks=range(img.shape[0]))
    rect = plt.Rectangle((1.5, 1.5), 3, 3, edgecolor="red", linewidth=3, fill=False)
    ax.add_patch(rect)
plt.show()

### <a id='toc2_4_2_'></a>[Example 2: 1D Convolution](#toc0_)


In [ ]:
signal_1d = np.array([0, 1, 2, 3, 4, 5, 4, 3, 2, 1, 0], dtype=np.float32)
filter_1d = np.array([1, 0, -1], dtype=np.float32)

# log
print(f"signal_1d:\n{signal_1d}\n")
print(f"filter_1d:\n{filter_1d}")

#### <a id='toc2_4_2_1_'></a>[Manual](#toc0_)


In [ ]:
def conv1d(signal: NDArray, kernel: NDArray) -> NDArray:
    signal_len = len(signal)
    kernel_len = len(kernel)
    output_len = signal_len + kernel_len - 1  # full convolution length

    kernel = kernel[::-1]
    padded_signal = np.pad(signal, (kernel_len - 1, kernel_len - 1), mode="constant")
    result = np.array([np.sum(padded_signal[i : i + kernel_len] * kernel) for i in range(output_len)])

    return result

In [ ]:
signal_1d

In [ ]:
signal_1d_conv_1 = conv1d(signal_1d, filter_1d)

# plot
titles = ["Input", "Filter", "Full"]
signals = [signal_1d, filter_1d, signal_1d_conv_1]
fig, axes = plt.subplots(1, 3, figsize=(12, 3), layout="compressed", sharex=True, sharey=True)
for col, (ax, title, data) in enumerate(zip(axes, titles, signals)):
    ax.stem(data, basefmt=" ")
    ax.set_title(title)
    ax.grid(True, linewidth=0.5, linestyle="--", color="gray")
plt.show()

#### <a id='toc2_4_2_2_'></a>[Using NumPy](#toc0_)


In [ ]:
signal_1d_conv_2 = np.convolve(signal_1d, filter_1d, mode="valid")
signal_1d_conv_3 = np.convolve(signal_1d, filter_1d, mode="same")
signal_1d_conv_4 = np.convolve(signal_1d, filter_1d, mode="full")

# plot
titles = ["Input", "Filter", "Valid", "Same", "Full"]
signals = [signal_1d, filter_1d, signal_1d_conv_2, signal_1d_conv_3, signal_1d_conv_4]
fig, axes = plt.subplots(1, 5, figsize=(24, 3), layout="compressed", sharex=True, sharey=True)
for col, (ax, title, data) in enumerate(zip(axes, titles, signals)):
    ax.stem(data, basefmt=" ")
    ax.set_title(title)
    ax.grid(True, linewidth=0.5, linestyle="--", color="gray")
plt.show()

#### <a id='toc2_4_2_3_'></a>[Using SciPy](#toc0_)


In [ ]:
signal_1d_conv_5 = sp.signal.convolve(signal_1d, filter_1d, mode="valid")
signal_1d_conv_6 = sp.signal.convolve(signal_1d, filter_1d, mode="same")
signal_1d_conv_7 = sp.signal.convolve(signal_1d, filter_1d, mode="full")

# plot
titles = ["Input", "Filter", "Valid", "Same", "Full"]
signals = [signal_1d, filter_1d, signal_1d_conv_5, signal_1d_conv_6, signal_1d_conv_7]
fig, axes = plt.subplots(1, 5, figsize=(20, 3), layout="compressed", sharex=True, sharey=True)
for col, (ax, title, data) in enumerate(zip(axes, titles, signals)):
    ax.stem(data, basefmt=" ")
    ax.set_title(title)
    ax.grid(True, linewidth=0.5, linestyle="--", color="gray")
plt.show()

### <a id='toc2_4_3_'></a>[Example 3: 2D Convolution](#toc0_)


In [ ]:
signal_2d = np.array([[1, 2, 3], [2, 1, 1], [3, 1, 2]], dtype=np.float32)
filter_2d = np.array([[3, 1, 2], [1, 2, 1], [3, 1, 1]], dtype=np.float32)

# log
print(f"signal_2d:\n{signal_2d}\n")
print(f"filter_2d:\n{filter_2d}")

#### <a id='toc2_4_3_1_'></a>[Manual](#toc0_)


In [ ]:
def conv2d(image: NDArray, kernel: NDArray) -> NDArray:
    image_h, image_w = image.shape
    kernel_h, kernel_w = kernel.shape
    output_h = image_h + kernel_h - 1  # full convolution height
    output_w = image_w + kernel_w - 1  # full convolution width
    kernel = np.flipud(np.fliplr(kernel))

    # pad the image with zeros
    pad_h, pad_w = kernel_h - 1, kernel_w - 1
    padded_image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode="constant", constant_values=0)

    # compute convolution
    result = np.zeros((output_h, output_w))
    for i in range(output_h):
        for j in range(output_w):
            result[i, j] = np.sum(padded_image[i : i + kernel_h, j : j + kernel_w] * kernel)

    return result

In [ ]:
signal_2d_conv_1 = conv2d(signal_2d, filter_2d)

# plot
signals = [signal_2d, filter_2d, signal_2d_conv_1]
titles = ["signal_2d", "filter_2d", "signal_2d_conv_1"]
fig, axes = plt.subplots(1, 3, figsize=(9, 3), layout="compressed")
for ax, signal, title in zip(axes, signals, titles):
    ax.imshow(signal, cmap="gray")
    ax.set(title=title, xticks=range(signal.shape[1]), yticks=range(signal.shape[0]))
plt.show()

#### <a id='toc2_4_3_2_'></a>[Using OpenCV](#toc0_)

- It performs convolution operations using the **same** mode by default.


In [ ]:
kernel_h, kernel_w = filter_2d.shape
pad_h, pad_w = kernel_h - 1 - 1, kernel_w - 1 - 1
signal_2d_padded = cv2.copyMakeBorder(signal_2d, pad_h, pad_h, pad_w, pad_w, borderType=cv2.BORDER_CONSTANT, value=0)
filter_2d_flip = np.flipud(np.fliplr(filter_2d))
signal_2d_conv_2 = cv2.filter2D(signal_2d_padded, ddepth=-1, kernel=filter_2d_flip, borderType=cv2.BORDER_CONSTANT)

# plot
signals = [signal_2d, filter_2d, signal_2d_conv_2]
titles = ["signal_2d", "filter_2d", "signal_2d_conv_2"]
fig, axes = plt.subplots(1, 3, figsize=(9, 3), layout="compressed")
for ax, signal, title in zip(axes, signals, titles):
    ax.imshow(signal, cmap="gray")
    ax.set(title=title, xticks=range(signal.shape[1]), yticks=range(signal.shape[0]))
plt.show()

#### <a id='toc2_4_3_3_'></a>[Using SciPy](#toc0_)


In [ ]:
signal_2d_conv_3 = sp.signal.convolve2d(signal_2d, filter_2d, mode="full", boundary="fill", fillvalue=0)

# plot
signals = [signal_2d, filter_2d, signal_2d_conv_3]
titles = ["signal_2d", "filter_2d", "signal_2d_conv_3"]
fig, axes = plt.subplots(1, 3, figsize=(9, 3), layout="compressed")
for ax, signal, title in zip(axes, signals, titles):
    ax.imshow(signal, cmap="gray")
    ax.set(title=title, xticks=range(signal.shape[1]), yticks=range(signal.shape[0]))
plt.show()

### <a id='toc2_4_4_'></a>[Example 4: Separable Convolution](#toc0_)


In [ ]:
signal_2d = np.array([[1, 2, 3], [2, 1, 1], [3, 1, 2]], dtype=np.float32)
filter_2d = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)

# log
print(f"signal_2d:\n{signal_2d}\n")
print(f"filter_2d:\n{filter_2d}")

#### <a id='toc2_4_4_1_'></a>[Manual](#toc0_)


In [ ]:
def separate_kernel(kernel: NDArray) -> tuple[np.ndarray, np.ndarray] | None:
    U, S, Vt = np.linalg.svd(kernel)
    S = np.sqrt(S[0])
    k_c = U[:, 0] * S
    k_r = Vt[0, :] * S

    reconstructed_kernel = np.outer(k_c, k_r)
    if not np.allclose(kernel, reconstructed_kernel, atol=1e-6):
        return None

    return k_r, k_c

In [ ]:
def conv2d_separable(image: NDArray, k_r: NDArray, k_c: NDArray) -> NDArray:
    image_h, image_w = image.shape
    k_r = np.flip(k_r)
    k_c = np.flip(k_c)
    pad_r = len(k_r) - 1
    pad_c = len(k_c) - 1

    padded_image = np.pad(image, ((0, 0), (pad_r, pad_r)), mode="constant", constant_values=0)
    intermediate = np.zeros((image_h, image_w + pad_r))
    for i in range(image_h):
        for j in range(image_w + pad_r):
            intermediate[i, j] = np.sum(padded_image[i, j : j + len(k_r)] * k_r)

    padded_intermediate = np.pad(intermediate, ((pad_c, pad_c), (0, 0)), mode="constant", constant_values=0)
    output_h, output_w = image_h + pad_c, image_w + pad_r
    output = np.zeros((output_h, output_w))
    for i in range(output_h):
        for j in range(output_w):
            output[i, j] = np.sum(padded_intermediate[i : i + len(k_c), j] * k_c)

    return output

In [ ]:
k_r, k_c = separate_kernel(filter_2d)
signal_2d_conv_4 = conv2d(signal_2d, filter_2d)
signal_2d_conv_5 = conv2d_separable(signal_2d, k_r, k_c)

# log
print(f"k_r                : {k_r}")
print(f"k_c                : {k_c}")
print(f"np.outer(k_c, k_r) :\n{np.outer(k_c, k_r)}")

In [ ]:
# plot
signals = [signal_2d, signal_2d_conv_4, signal_2d_conv_5]
titles = ["signal_2d", "2D Convolution", "Separable Convolution"]
fig, axes = plt.subplots(1, 3, figsize=(9, 3), layout="compressed")
for ax, signal, title in zip(axes, signals, titles):
    ax.imshow(signal, cmap="gray")
    ax.set(title=title, xticks=range(signal.shape[1]), yticks=range(signal.shape[0]))
plt.show()

### <a id='toc2_4_5_'></a>[Example 5: 2D Cross-Correlation](#toc0_)


In [ ]:
signal_2d = np.array([[1, 2, 3], [2, 1, 1], [3, 1, 2]], dtype=np.float32)
filter_2d = np.array([[3, 1, 2], [1, 2, 1], [3, 1, 1]], dtype=np.float32)

# log
print(f"signal_2d:\n{signal_2d}\n")
print(f"filter_2d:\n{filter_2d}")

#### <a id='toc2_4_5_1_'></a>[Using SciPy](#toc0_)

In [ ]:
signal_2d_corr_1 = sp.signal.correlate2d(signal_2d, filter_2d, mode="full", boundary="fill", fillvalue=0)

# plot
signals = [signal_2d, filter_2d, signal_2d_corr_1]
titles = ["signal_2d", "filter_2d", "signal_2d_corr_1"]
fig, axes = plt.subplots(1, 3, figsize=(9, 3), layout="compressed")
for ax, signal, title in zip(axes, signals, titles):
    ax.imshow(signal, cmap="gray")
    ax.set(title=title, xticks=range(signal.shape[1]), yticks=range(signal.shape[0]))
plt.show()